In [1]:
! pip install rdkit


[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


## RDFingerprints

In [ ]:
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import rdFingerprintGenerator
import numpy as np

def smiles_to_ecfp4(smiles, n_bits=2048):
    """
    Converts a SMILES string to an ECFP4 fingerprint.
    
    Args:
        smiles (str): The SMILES string of a molecule.
        n_bits (int): Length of the fingerprint vector (default is 2048).
    
    Returns:
        np.array: A binary fingerprint array.
    """
    try:
        # Convert SMILES to RDKit molecule
        mol = Chem.MolFromSmiles(smiles)
        if mol is None:
            raise ValueError("Invalid SMILES")
        
        # Generate ECFP4 fingerprint
        mfpgen = rdFingerprintGenerator.GetMorganGenerator(fpSize=n_bits)
        morgan_fp = mfpgen.GetFingerprint(mol)
        
        # Convert to numpy array
        arr = np.zeros((n_bits,), dtype=int)
        Chem.DataStructs.ConvertToNumpyArray(morgan_fp, arr)
        return arr
    except Exception as e:
        print(f"Error processing SMILES {smiles}: {e}")
        return None
    
     
# feature_matrix = np.vstack(fingerprints)


NameError: name 'fingerprints' is not defined

## chemBERTa

In [9]:
from transformers import RobertaTokenizer, TFRobertaModel
import tensorflow as tf

# Load the chemBERTa model and tokenizer
tokenizer = RobertaTokenizer.from_pretrained("seyonec/PubChem10M_SMILES_BPE_450k")
model = TFRobertaModel.from_pretrained("seyonec/PubChem10M_SMILES_BPE_450k", from_pt=True)

def smiles_to_chemberta_embedding(smiles):
    """
    Converts a SMILES string to a chemBERTa embedding.
    
    Args:
        smiles (str): The SMILES string of a molecule.
    
    Returns:
        np.array: The chemBERTa embedding.
    """
    try:
        # Tokenize the SMILES string
        inputs = tokenizer(smiles, return_tensors="tf")
        
        # Generate embeddings
        outputs = model(inputs)
        
        # Get the embeddings from the last hidden state
        embeddings = tf.reduce_mean(outputs.last_hidden_state, axis=1).numpy()
        return embeddings
    except Exception as e:
        print(f"Error processing SMILES {smiles}: {e}")
        return None



Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaModel: ['lm_head.decoder.bias', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'roberta.embeddings.position_ids', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing TFRobertaModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFRobertaModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


# Data Prep

## chemBERTa prep

In [10]:
import pandas as pd
import numpy as np

data = pd.read_csv('SyntheticData.csv')
# data.head()
input = data['Smile']
output = data['Heat of Combustion']

# Convert SMILES to chemBERTa embeddings
chemberta_embeddings = [smiles_to_chemberta_embedding(s) for s in input]

# Convert to numpy array
feature_matrix = np.vstack(chemberta_embeddings)
print(feature_matrix)

test_data = pd.read_csv('testing_data.csv')
# test_data.head()
test_input = test_data['Smile']
test_output = test_data['Heat of Combustion']

# Convert test SMILES to chemBERTa embeddings
test_chemberta_embeddings = [smiles_to_chemberta_embedding(s) for s in test_input]
# Print the size of one of the embeddings
print(f"Size of one embedding: {test_chemberta_embeddings[0].shape}")

# Convert to numpy array
test_feature_matrix = np.vstack(test_chemberta_embeddings)
print(test_feature_matrix)

# Write the feature_matrix to a file
# np.save('embeddings/chemBertaEmbeddings.npy', feature_matrix)


[[-1.4256042  -1.3949604   1.5881201  ... -0.00274044  0.25688475
  -0.29546866]
 [-1.4463953  -1.7581638   0.9259674  ... -0.88145083  0.28398564
  -0.0378332 ]
 [-1.9992245  -1.5124565   1.2577149  ... -1.4636784   0.14179237
   0.27983335]
 ...
 [-0.41555628 -0.8626808   1.0436213  ... -0.1684835   0.21819799
  -0.26750368]
 [-0.637986   -0.19990233  1.4843488  ... -1.1413604   0.8553332
  -0.59937996]
 [ 0.22246566 -0.5423054   0.8084985  ...  0.28055328  0.30308014
   0.46292472]]
Size of one embedding: (1, 768)
[[-1.4639498  -2.090591    1.9159063  ...  0.4178756   0.24905889
  -0.26775038]
 [-2.075873   -1.5415013   1.9413147  ...  0.35351324  0.32953373
   0.0907841 ]
 [-2.6733763  -1.445852    1.6646048  ...  0.8060034   0.0337576
   0.11659135]
 ...
 [-0.34806222 -0.06990272  0.31357646 ... -0.12339465 -0.10441314
  -0.31312   ]
 [ 0.11035563 -0.12319657  0.22988446 ...  0.104612    0.3350938
  -0.47396162]
 [-0.30993482 -0.6233391  -0.02259463 ... -1.0125186   0.70143855
  -

# Models

## Load prefered embeddings

In [ ]:
import pandas as pd
import numpy as np
data = pd.read_csv('clean_full_combustion_with_smile.csv')
# data.head()
input = data['Smile']
output = data['Heat of Combustion']
# Choose embedding type
# feature_matrix = np.load('embeddings/ecfp4Embeddings.npy')
feature_matrix = np.load('embeddings/chemBertaEmbeddings.npy')

## Random Forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(feature_matrix, output, test_size=0.2, random_state=42)

# Define the hyperparameters directly
best_params = {'max_depth': 20, 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 100}

# Create the RandomForestRegressor model with the best parameters
best_model = RandomForestRegressor(**best_params)

# Train the best model
model = best_model
model.fit(X_train, y_train)

# Evaluate the model
predictions = model.predict(X_test)
mse = mean_squared_error(y_test, predictions)
r2 = model.score(X_test, y_test)
print(f"R-squared: {r2}")

mae = np.mean(np.abs(predictions - y_test))
print(f"Mean Absolute Error: {mae}")
print(f"Mean Squared Error: {mse}")


Best parameters: {'max_depth': 20, 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 100}
R-squared: 0.09610684852917839
Mean Absolute Error: 1896.836579739843
Mean Squared Error: 7353301.653098491


## XGBoost 
Consistently the lowest accuracy

In [ ]:
from xgboost import XGBRegressor
from sklearn.model_selection import KFold

# Define the best parameters
best_params = {'learning_rate': 0.01, 'max_depth': 9, 'n_estimators': 200, 'subsample': 0.8}

# Create the XGBRegressor model with the best parameters
best_model = XGBRegressor(**best_params)
# Perform k-fold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)
mse_scores = []
r2_scores = []

for train_index, val_index in kf.split(X_train):
    X_tr, X_val = X_train[train_index], X_train[val_index]
    y_tr, y_val = y_train.iloc[train_index], y_train.iloc[val_index]
    
    best_model.fit(X_tr, y_tr)
    val_predictions = best_model.predict(X_val)
    
    mse_scores.append(mean_squared_error(y_val, val_predictions))
    r2_scores.append(best_model.score(X_val, y_val))

print(f"Average MSE: {np.mean(mse_scores)}")
print(f"Average R-squared: {np.mean(r2_scores)}")

Average MSE: 12084850.46010613
Average R-squared: -0.04456005414072426


## SVM
Fast and highest accuracy so far

In [ ]:
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

# Apply the best parameters found from grid search directly to the SVR model
best_params = {'C': 10, 'degree': 2, 'epsilon': 0.5, 'gamma': 'scale', 'kernel': 'linear'}
best_model = SVR(**best_params)

# Perform k-fold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)
mse_scores = []
mae_scores = []
r2_scores = []

for train_index, val_index in kf.split(feature_matrix):
    X_tr, X_val = feature_matrix[train_index], feature_matrix[val_index]
    y_tr, y_val = output.iloc[train_index], output.iloc[val_index]
    
    best_model.fit(X_tr, y_tr)
    val_predictions = best_model.predict(X_val)
    
    mse_scores.append(mean_squared_error(y_val, val_predictions))
    mae_scores.append(np.mean(np.abs(val_predictions - y_val)))
    r2_scores.append(best_model.score(X_val, y_val))

print(f"Average MSE: {np.mean(mse_scores)}")
print(f"Average MAE: {np.mean(mae_scores)}")
print(f"Average R-squared: {np.mean(r2_scores)}")


Average MSE: 9525979.88032572
Average MAE: 1754.5031615690343
Average R-squared: 0.17999267181989503


## Neural Network
Needs work


In [14]:
from keras.models import Sequential
from keras.layers import Conv1D, Flatten, Dense, Input, Dropout
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import numpy as np

# Define the CNN model
def create_cnn_model(input_shape):
    model = Sequential()
    model.add(Input(shape=input_shape))
    # test dense compared to convolutoinal
    model.add(Dense(32, activation='relu'))
    # model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1, activation='linear'))
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')
    return model

# Perform k-fold cross-validation
kf = KFold(n_splits=7, shuffle=True, random_state=42)
mse_scores = []
mae_scores = []

for train_index, val_index in kf.split(feature_matrix):
    X_tr, X_val = feature_matrix[train_index], feature_matrix[val_index]
    y_tr, y_val = output.iloc[train_index], output.iloc[val_index]
    
    # Reshape the input data to fit the CNN model
    X_tr = X_tr.reshape((X_tr.shape[0], X_tr.shape[1], 1))
    X_val = X_val.reshape((X_val.shape[0], X_val.shape[1], 1))
    
    # Create and train the CNN model
    model = create_cnn_model((X_tr.shape[1], 1))
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    model.fit(X_tr, y_tr, epochs=10000, batch_size=64, validation_data=(X_val, y_val), callbacks=[early_stopping], verbose=0)
    
    # Evaluate the model
    val_predictions = model.predict(X_val)
    mse_scores.append(mean_squared_error(y_val, val_predictions))
    # mae_scores.append(np.mean(np.abs(val_predictions - y_val)))

print(f"Average MSE: {np.mean(mse_scores)}")
# print(f"Average MAE: {np.mean(mae_scores)}")
# Calculate R-squared value
r2_scores = [model.evaluate(X_val, y_val, verbose=0) for train_index, val_index in kf.split(feature_matrix)]
average_r2 = 1 - np.mean(r2_scores) / np.var(output)
print(f"Average R-squared: {average_r2}")
print("-----------------------------------")
model.summary()

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
Average MSE: 2788282.065055718
Average R-squared: 0.8672423322067074
-----------------------------------


Model: "sequential_27"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_202 (Dense)               │ (None, 768, 32)        │            64 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_27 (Flatten)            │ (None, 24576)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_203 (Dense)               │ (None, 64)             │     1,572,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_81 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_204 (Dense)               │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_205 (Dense)               │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_82 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_206 (Dense)               │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_83 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_207 (Dense)               │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_208 (Dense)               │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_209 (Dense)               │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,781,573 (18.24 MB)

 Trainable params: 1,593,857 (6.08 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 3,187,716 (12.16 MB)

In [16]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Reshape the test feature matrix for the CNN model
test_feature_matrix_reshaped = test_feature_matrix.reshape((test_feature_matrix.shape[0], test_feature_matrix.shape[1], 1))

# Make predictions on the test set
test_predictions = model.predict(test_feature_matrix_reshaped)

# Calculate evaluation metrics
mse_test = mean_squared_error(test_output, test_predictions)
mae_test = mean_absolute_error(test_output, test_predictions)
r2_test = r2_score(test_output, test_predictions)

print(f"Test MSE: {mse_test}")
print(f"Test MAE: {mae_test}")
print(f"Test R-squared: {r2_test}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
Test MSE: 4984136.111082153
Test MAE: 1633.4812698216933
Test R-squared: 0.2304650778791445


# Training Data vs Synthetic Data Results

| Metric                | Neural Net (Training Data) | Neural Net (Synthetic Data) | CNN (Training Data) | CNN (Synthetic Data) |
|-----------------------|----------------------------|-----------------------------|----------------------|-----------------------|
| **Average MSE**       | 9,794,885        | 2,788,282         | 9,780,797  | 2,500,507   |
| **Average R-squared** | 0.7848886029948038        | 0.8672423322067074          | 0.7871524146612794   | 0.7930993420803031    |
| **Test MSE**          | 4,700,471       | 4,984,136         | 4,624,448  | 4,913,290   |
| **Test MAE**          | 1,767       | 1,633         | 1,713   | 1,558   |
| **Test R-squared**    | 0.2742619564594171        | 0.2304650778791445          | 0.28599978637983003  | 0.2414033579524828    |


In [3]:
def get_test_example(sample_index):
    print("Sample Index:", sample_index, "of", len(test_input))

    # Get the example input, prediction, and target
    example_input = test_input.iloc[sample_index]
    example_prediction = test_predictions[sample_index][0]
    example_target = test_output.iloc[sample_index]
    # Calculate the difference
    difference = abs(example_prediction - example_target)

    # Print the results
    print(f"Testing Input (SMILES): {example_input}")
    print(f"Model Prediction: {example_prediction:.2f}")
    print(f"Actual Value: {example_target:.2f}")
    print(f"Difference: {difference:.2f}")




In [4]:
# Select an example index from the test set
example_index = 15  # You can change this index to test other samples
get_test_example(example_index)

NameError: name 'test_input' is not defined

In [ ]:
import datetime
from tensorflow.keras.callbacks import TensorBoard
from sklearn.model_selection import train_test_split



# Create a logs directory using current timestamp
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

# (Re)create and train the model with the TensorBoard callback,
# so that training logs get saved.
cnn_model = create_cnn_model((feature_matrix.shape[1], 1))
X_train, X_test, y_train, y_test = train_test_split(feature_matrix, output, test_size=0.2, random_state=42)
X_train_cnn = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test_cnn = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

cnn_model.fit(
    X_train_cnn, y_train,
    batch_size=64,
    epochs=100,
    validation_data=(X_test_cnn, y_test),
    callbacks=[tensorboard_callback],
    verbose=1
)
%load_ext tensorboard
# Launch tensorboard (adjust the --logdir path if needed)
%tensorboard --logdir logs/fit

Epoch 1/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 180ms/step - loss: 18189822.0000 - val_loss: 9017719.0000
Epoch 2/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step - loss: 18406246.0000 - val_loss: 8968423.0000
Epoch 3/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step - loss: 12757139.0000 - val_loss: 8900638.0000
Epoch 4/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step - loss: 12173156.0000 - val_loss: 8809048.0000
Epoch 5/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step - loss: 13690339.0000 - val_loss: 8690604.0000
Epoch 6/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step - loss: 14152281.0000 - val_loss: 8543607.0000
Epoch 7/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step - loss: 17652630.0000 - val_loss: 8363805.0000
Epoch 8/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step - loss: 18108696.0000 - val_loss: 8166584.5000
Epoch 9/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step - loss: 11102976.0000 - val_loss: 7982208.5000
Epoch 10/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step - loss: 9427222.0000 - val_loss: 7858060.5000
Epoch 11/100
4/4 ━━━━━━

Reusing TensorBoard on port 6006 (pid 26504), started 0:14:20 ago. (Use '!kill 26504' to kill it.)

In [ ]:
# Select an example index
example_index = 400

# Get the example input, prediction, and target
example_input = feature_matrix[example_index]
example_prediction = model.predict(example_input.reshape(1, -1, 1))
example_target = output.iloc[example_index]
smile = input.iloc[example_index]
print(f"Example Input: {example_input}")
print(f"SMILES: {smile}")
print(f"Prediction: {example_prediction[0][0]}")
print(f"Target: {example_target}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
Example Input: [-1.30978394e+00 -1.30010307e+00  8.25705230e-01  3.31037372e-01
 -3.08806896e-01  4.08945709e-01  1.14933610e+00 -3.83327529e-02
 -6.71126723e-01  3.53949487e-01 -1.69453546e-01 -1.22124732e+00
 -3.96527797e-01 -1.14916480e+00  3.58762175e-01  3.78776878e-01
  9.05400634e-01  1.94160819e+00  8.04660797e-01 -1.32225335e+00
 -4.19817045e-02 -1.97264537e-01 -1.58748552e-01  6.07734859e-01
 -1.77657396e-01  5.08582115e-01 -7.63357580e-01 -7.50062943e-01
 -4.75394666e-01  1.12984478e+00 -6.08671606e-01  1.91492010e-02
 -1.38749886e+00  6.22744739e-01  7.81456649e-01 -9.65853155e-01
 -6.87889218e-01 -1.05017543e+00 -2.14500025e-01  6.43548369e-01
 -1.96617448e+00  1.84712291e-01  3.60025853e-01 -6.28009260e-01
  1.37049997e+00  1.18236184e+00  1.10573542e+00 -7.95287132e-01
  5.20733476e-01  6.74404681e-01 -2.33746767e-01  1.00349069e+00
 -5.13061345e-01  8.07707846e-01 -1.46388412e-01  7.93241024e-01
 -1.13919771e+00  2.70261705e-01  5.1

In [ ]:
from IPython.display import display

# First, reshape the feature matrix as required by the CNN model
X_all = feature_matrix.reshape((feature_matrix.shape[0], feature_matrix.shape[1], 1))

# Generate predictions for all samples using the previously trained model
preds = model.predict(X_all).flatten()

# Calculate the absolute discrepancy between predictions and targets
discrepancy = np.abs(preds - output.to_numpy())

# Create a new DataFrame with SMILES, Prediction, Target, and Discrepancy columns
results_df = pd.DataFrame({
    'SMILES': input,
    'Prediction': preds,
    'Target': output,
    'Discrepancy': discrepancy
})

results_df = results_df.sort_values(by='Discrepancy', ascending=False)
# Display the resulting table
display(results_df.head(10))
display(results_df.tail(10))

27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step


,SMILES,Prediction,Target,Discrepancy
471,C(CCC(=O)O)CC(=O)O,1067.860962,4863.73,3795.869038
756,C(CCC(=O)O)CC(=O)O,1067.860962,4863.73,3795.869038
186,C(CCC(=O)O)CC(=O)O,1067.860962,4863.73,3795.869038
554,CC1C2CCC1CC(C2)OC(=O)C3=CC(=CC(=C3)Cl)Cl,-38064.386719,-35099.60,2964.786719
839,CC1C2CCC1CC(C2)OC(=O)C3=CC(=CC(=C3)Cl)Cl,-38064.386719,-35099.60,2964.786719
269,CC1C2CCC1CC(C2)OC(=O)C3=CC(=CC(=C3)Cl)Cl,-38064.386719,-35099.60,2964.786719
167,C1=NC(=C(N1)C(=O)O)N,1041.532104,-1910.62,2952.152104
737,C1=NC(=C(N1)C(=O)O)N,1041.532104,-1910.62,2952.152104
452,C1=NC(=C(N1)C(=O)O)N,1041.532104,-1910.62,2952.152104
563,C1=CC=C2C(=C1)C3=CC=CC=C3N2,7133.939453,10040.60,2906.660547


,SMILES,Prediction,Target,Discrepancy
499,C1=CC(=C[N+](=C1)C2C(C(C(O2)COP(=O)(O)O)O)O)C(...,-709.552795,-715.90,6.347205
433,C(CN)C=O,4164.063477,4159.50,4.563477
148,C(CN)C=O,4164.063477,4159.50,4.563477
718,C(CN)C=O,4164.063477,4159.50,4.563477
400,N#CC#N,-1093.251465,-1097.07,3.818535
115,N#CC#N,-1093.251465,-1097.07,3.818535
685,N#CC#N,-1093.251465,-1097.07,3.818535
388,NC,-1088.399658,-1086.81,1.589658
103,NC,-1088.399658,-1086.81,1.589658
673,NC,-1088.399658,-1086.81,1.589658


```flowchart TD
    A[Input Layer<br/>(input_shape)]
    B[Conv1D<br/>32 filters, kernel size=3<br/>(ReLU)]
    C[Flatten]
    D[Dense Layer<br/>128 units, ReLU]
    E[Dropout<br/>(rate = 0.3)]
    F[Dense Layer<br/>128 units, ReLU]
    G[Dense Layer<br/>1 unit, Linear]
    
    A --> B
    B --> C
    C --> D
    D --> E
    E --> F
    F --> G```


# Naive Bias ?